# MLP Autoencoder

In [1]:
import os
import tensorflow as tf
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)  ## just for ignore DeprcationWarning message
print("Current version [%s]" %(tf.__version__))
print("Packages Loaded")

Current version [1.3.1]
Packages Loaded


### Configurations

In [2]:
# Dataset Configurations
tf.app.flags.DEFINE_integer('img_size', 28, """Image size of MNIST dataset""")

# Network Configurations
tf.app.flags.DEFINE_integer('batch_size', 100, """Number of images to process in a batch""")
tf.app.flags.DEFINE_float('l1_ratio', 0.5, """Ratio of level1""")
tf.app.flags.DEFINE_float('l2_ratio', 0.5, """Ratio of level2""")

# Optimization Configurations
tf.app.flags.DEFINE_float('lr', 0.001, """Learning rate""")

# Training Configurations
tf.app.flags.DEFINE_integer('training_epochs', 500, """Number of epochs to run""")
tf.app.flags.DEFINE_integer('display_step', 5, """Number of iterations to display training output""")
tf.app.flags.DEFINE_integer('save_step', 5, """Number of interations to save checkpoint""")
tf.app.flags.DEFINE_integer('save_max', 5, """Number of checkpoints to remain""")


# Save Configurations
tf.app.flags.DEFINE_string('nets', './nets', """Directory where to write the checkpoints""")
tf.app.flags.DEFINE_string('outputs', './outputs', """Directory where to save the output images""")
tf.app.flags.DEFINE_string('tboard', './tensorboard', """Directory where to save the tensorboard logs""")


FLAGS = tf.app.flags.FLAGS
print("FLAGS READY")

FLAGS READY


### GPU control

In [3]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True

### Load Data

In [4]:
mnist = input_data.read_data_sets('../../data/', one_hot=True)
train_img = mnist.train.images
train_label = mnist.train.labels
test_img = mnist.test.images
test_label = mnist.test.labels
print("MNIST ready")

Extracting ../../data/train-images-idx3-ubyte.gz
Extracting ../../data/train-labels-idx1-ubyte.gz
Extracting ../../data/t10k-images-idx3-ubyte.gz
Extracting ../../data/t10k-labels-idx1-ubyte.gz
MNIST ready


### Generating random noise mask

In [5]:
def noise_mask(prob=0.5):
    mask = np.zeros([FLAGS.img_size, FLAGS.img_size])
    rd = np.random.random()
    if rd > prob:
        # threshold of the size of masks
        uthd = FLAGS.img_size    
        lthd = 0     
        # mask size should be beween 14x14, 5x5
        while(uthd>14 or lthd<5):
            ver1 = np.random.random_integers(0, FLAGS.img_size-1, size= 2)   # vertex1
            ver2 = np.random.random_integers(0, FLAGS.img_size-1, size= 2)    # vertex2
            uthd = np.maximum(np.abs(ver1[0]-ver2[0]), np.abs(ver1[1]-ver2[1]))    # upperbound
            lthd = np.minimum(np.abs(ver1[0]-ver2[0]), np.abs(ver1[1]-ver2[1]))    # lowerbound
        xmin = np.minimum(ver1[0], ver2[0])    # left x value
        xmax = np.maximum(ver1[0], ver2[0])    # right x value
        ymin = np.minimum(ver1[1], ver2[1])    # top y value
        ymax = np.maximum(ver1[1], ver2[1])    # bottom y value
        noise = np.random.random((xmax-xmin+1, ymax-ymin+1))    # random sample in [0,1]
        mask[xmin:xmax+1, ymin:ymax+1] = noise    # noise mask with location
        mask_meta = [xmin, xmax, ymin, ymax, noise, mask]
    mask = np.reshape(mask, [-1])
    return mask

def noise_batch(batch_num):
    # make random noise batch
    mask_batch = np.zeros([batch_num, FLAGS.img_size*FLAGS.img_size])
    for i in range(batch_num):
        mask_batch[i,:] = noise_mask()
    return mask_batch

### Occlusion generation

In [6]:
def occl(target, disturb):
    mask = (disturb==0).astype(float)
    masked_target = np.multiply(target, mask)
    crpt = np.add(masked_target, disturb)
    return crpt

### Nested MLP

In [7]:
def _nested_mlp(l1, l2_s, l2, out_channel, name, stddev=0.1, is_init=False, is_last=False):
    l1_shape = l1.get_shape()[1]
    l2_shape = l2.get_shape()[1]
    l2_s_shape = l2_s.get_shape()[1]
    
    if is_init:
        # input is the input image
        with tf.device('/CPU:0'):
            with tf.variable_scope('level1'):
                with tf.variable_scope(name):
                    l1_weights = tf.get_variable('weights', 
                                                 [l1_shape, out_channel*FLAGS.l1_ratio], 
                                                 tf.float32, 
                                                 initializer=tf.random_normal_initializer(stddev=stddev))
                    l1_biases = tf.get_variable('biases', 
                                                [out_channel*FLAGS.l1_ratio],
                                                tf.float32, 
                                                initializer=tf.random_normal_initializer(stddev=stddev))
            with tf.variable_scope('level2'):
                with tf.variable_scope(name):
                    l2_s_weights = tf.get_variable('weights_shell', 
                                                 [l2_s_shape, out_channel*FLAGS.l2_ratio], 
                                                 tf.float32, 
                                                 initializer=tf.random_normal_initializer(stddev=stddev))
                    l2_s_biases = tf.get_variable('biases_shell', 
                                                [out_channel*FLAGS.l2_ratio],
                                                tf.float32, 
                                                initializer=tf.random_normal_initializer(stddev=stddev))
                    

        l1_mlp = tf.nn.sigmoid(tf.add(tf.matmul(l1, l1_weights), l1_biases))
        l2_s_mlp = tf.nn.sigmoid(tf.add(tf.matmul(l2_s, l2_s_weights), l2_s_biases))
        l2_mlp = tf.concat((l1_mlp, l2_s_mlp), 1)
    
    elif is_last:
        # output is the generated image
        with tf.device('/CPU:0'):
            with tf.variable_scope('level1'):
                with tf.variable_scope(name):
                    l1_weights = tf.get_variable('weights', 
                                                 [l1_shape, out_channel], 
                                                 tf.float32, 
                                                 initializer=tf.random_normal_initializer(stddev=stddev))
                    l1_biases = tf.get_variable('biases', 
                                                [out_channel],
                                                tf.float32, 
                                                initializer=tf.random_normal_initializer(stddev=stddev))
            with tf.variable_scope('level2'):
                with tf.variable_scope(name):
                    l2_s_weights = tf.get_variable('weights_shell', 
                                                 [l2_s_shape, out_channel], 
                                                 tf.float32, 
                                                 initializer=tf.random_normal_initializer(stddev=stddev))
                    l2_s_biases = tf.get_variable('biases_shell', 
                                                [out_channel],
                                                tf.float32, 
                                                initializer=tf.random_normal_initializer(stddev=stddev))
                    l2_weights = tf.get_variable('weights', 
                                                 [l2_shape, out_channel], 
                                                 tf.float32, 
                                                 initializer=tf.random_normal_initializer(stddev=stddev))
                    l2_biases = tf.get_variable('biases', 
                                                [out_channel],
                                                tf.float32, 
                                                initializer=tf.random_normal_initializer(stddev=stddev))
                    

        l1_mlp = tf.nn.sigmoid(tf.add(tf.matmul(l1, l1_weights), l1_biases))
        l2_s_mlp = tf.nn.sigmoid(tf.add(tf.matmul(l2_s, l2_s_weights), l2_s_biases))
        l2_mlp = tf.nn.sigmoid(tf.add(tf.matmul(l2, l2_weights), l2_biases))
                                 
    else:
        with tf.device('/CPU:0'):
            with tf.variable_scope('level1'):
                with tf.variable_scope(name):
                    l1_weights = tf.get_variable('weights', 
                                                 [l1_shape, out_channel*FLAGS.l1_ratio], 
                                                 tf.float32, 
                                                 initializer=tf.random_normal_initializer(stddev=stddev))
                    l1_biases = tf.get_variable('biases', 
                                                [out_channel*FLAGS.l1_ratio],
                                                tf.float32, 
                                                initializer=tf.random_normal_initializer(stddev=stddev))
            with tf.variable_scope('level2'):
                with tf.variable_scope(name):
                    l2_s_weights = tf.get_variable('weights_shell', 
                                                   [l2_s_shape, out_channel*FLAGS.l2_ratio], 
                                                   tf.float32, 
                                                   initializer=tf.random_normal_initializer(stddev=stddev))
                    l2_s_biases = tf.get_variable('biases_shell', 
                                                  [out_channel*FLAGS.l2_ratio],
                                                  tf.float32, 
                                                  initializer=tf.random_normal_initializer(stddev=stddev))
                    l2_weights_1 = tf.get_variable('weights_1', 
                                                   [l2_s_shape, out_channel*FLAGS.l1_ratio], 
                                                   tf.float32, 
                                                   initializer=tf.random_normal_initializer(stddev=stddev))
                    l2_biases_1 = tf.get_variable('biases_1', 
                                                  [out_channel*FLAGS.l1_ratio],
                                                  tf.float32, 
                                                  initializer=tf.random_normal_initializer(stddev=stddev))
                    l2_weights_2 = tf.get_variable('weights_2', 
                                                   [l1_shape, out_channel*FLAGS.l2_ratio], 
                                                   tf.float32, 
                                                   initializer=tf.random_normal_initializer(stddev=stddev))
                    l2_biases_2 = tf.get_variable('biases_2', 
                                                  [out_channel*FLAGS.l2_ratio],
                                                  tf.float32, 
                                                  initializer=tf.random_normal_initializer(stddev=stddev))

        l1_mlp_r = tf.add(tf.matmul(l1, l1_weights), l1_biases)
        l1_mlp = tf.nn.sigmoid(l1_mlp_r)
        
        l2_s_mlp = tf.nn.sigmoid(tf.add(tf.matmul(l2_s, l2_s_weights), l2_s_biases))
        
        l2_mlp_1_r = tf.add(tf.matmul(l2[:,l1_shape:l2_shape], l2_weights_1), l2_biases_1)
        l2_mlp_1 = tf.nn.sigmoid(tf.add(l1_mlp_r, l2_mlp_1_r))
        l2_mlp_2_r = tf.add(tf.matmul(l2[:,:l1_shape], l2_weights_2), l2_biases_2)
        l2_mlp_3_r = tf.add(tf.matmul(l2[:,l1_shape:l2_shape], l2_s_weights), l2_s_biases)
        l2_mlp_2 = tf.nn.sigmoid(tf.add(l2_mlp_2_r, l2_mlp_3_r))
        l2_mlp = tf.concat((l2_mlp_1, l2_mlp_2), 1)
        
        
    return l1_mlp, l2_s_mlp, l2_mlp

### Graph setup

In [8]:
# Network Topology
n_input = FLAGS.img_size*FLAGS.img_size
n_enc1 = 1024
n_enc2 = 512
n_enc3 = 256
n_dec1 = 512
n_dec2 = 1024
n_out = 784

# Inputs and Outputs
ph_pure = tf.placeholder("float", [None, n_input])    # pure image --- core
ph_noise= tf.placeholder("float", [None, n_input])    # noise --- shell1
ph_crpt = tf.placeholder("float", [None, n_input])    # corrupted image   --- level2


# Model
def nested_ae_mlp(_X):
    l1_enc1, l2_s_enc1, l2_enc1 = _nested_mlp(_X, _X, _X, n_enc1, name='enc1', is_init=True)
    l1_enc2, l2_s_enc2, l2_enc2 = _nested_mlp(l1_enc1, l2_s_enc1, l2_enc1, n_enc2, name='enc2')
    l1_enc3, l2_s_enc3, l2_enc3 = _nested_mlp(l1_enc2, l2_s_enc2, l2_enc2, n_enc3, name='enc3')
    l1_dec1, l2_s_dec1, l2_dec1 = _nested_mlp(l1_enc3, l2_s_enc3, l2_enc3, n_dec1, name='dec1')
    l1_dec2, l2_s_dec2, l2_dec2 = _nested_mlp(l1_dec1, l2_s_dec1, l2_dec1, n_dec2, name='dec2')
    l1_out, l2_s_out, l2_out = _nested_mlp(l1_dec2, l2_s_dec2, l2_dec2, n_out, name='out',is_last=True)
    return l1_out, l2_s_out, l2_out

# Generation
core_gen, shell2_gen, full_gen = nested_ae_mlp(ph_crpt)   # [None, n_input]

# Loss & Optimizer
with tf.name_scope("loss") as scope:
    loss = tf.reduce_mean(tf.nn.l2_loss(full_gen-ph_crpt)) + 2*tf.reduce_mean(tf.nn.l2_loss(core_gen-ph_pure))\
            + tf.reduce_mean(tf.nn.l2_loss(shell2_gen-ph_noise))
    _train_loss = tf.summary.scalar("train_loss", loss)
    _test_loss = tf.summary.scalar("test_loss", loss)

optm = tf.train.AdamOptimizer(learning_rate=FLAGS.lr).minimize(loss)


print("Graphs Ready")

Graphs Ready


### Initialize

In [9]:
merged = tf.summary.merge_all()
tensorboard_path = FLAGS.tboard
if not os.path.exists(tensorboard_path):
    os.makedirs(tensorboard_path)
writer = tf.summary.FileWriter(tensorboard_path)
init = tf.global_variables_initializer()

print("Initialize Ready")

Initialize Ready


### Data saving

In [10]:
outputdir = FLAGS.outputs
if not os.path.exists(outputdir+'/train'):
    os.makedirs(outputdir+'/train')

if not os.path.exists(outputdir+'/test'):
    os.makedirs(outputdir+'/test')
    
savedir = FLAGS.nets
if not os.path.exists(savedir):
    os.makedirs(savedir)
    
saver = tf.train.Saver(max_to_keep=FLAGS.save_max)
print("Saver ready")

Saver ready


### Run

In [11]:
# Parameters
training_epochs = FLAGS.training_epochs
batch_size = FLAGS.batch_size
display_step = FLAGS.display_step
# Plot
n_plot = 5    # plot 5 images
train_disp_idx = np.random.randint(mnist.train.num_examples, size=n_plot)    # fixed during train time
test_disp_idx = np.random.randint(mnist.test.num_examples, size=n_plot)

# Initialize
sess = tf.Session(config=config)
sess.run(init)

# Optimize
for epoch in range(training_epochs):
    total_cost = 0.
    n_total_batch = int(mnist.train.num_examples/batch_size)
    rand_train_idx = np.random.randint(mnist.train.num_examples, size=batch_size)
    rand_test_idx = np.random.randint(mnist.test.num_examples, size=batch_size)
    
    # Iteration
    for i in range(n_total_batch):
        batch_pure, _ = mnist.train.next_batch(batch_size)    # pure image
        noise = noise_batch(batch_size)    # random noise
        batch_crpt = occl(batch_pure, noise)   # corrupted image 
        feeds = {ph_pure: batch_pure, ph_noise: noise, ph_crpt: batch_crpt}
        sess.run(optm, feed_dict=feeds)
        #total_cost += sess.run(loss, feed_dict=feeds)
    #total_cost = total_cost / mnist.train.num_examples
    
    train_pure = train_img[rand_train_idx]    # pure image
    train_noise = noise_batch(batch_size)    # random noise
    train_crpt = occl(train_pure,train_noise)   # corrupted image
    train_feeds = {ph_pure: train_pure, ph_noise: train_noise, ph_crpt: train_crpt}
    train_loss, tb_train_loss = sess.run([loss,_train_loss], feed_dict=train_feeds)
    
    test_pure = test_img[rand_test_idx]    # pure image
    test_noise = noise_batch(batch_size)    # random noise
    test_crpt = occl(test_pure,test_noise)   # corrupted image
    test_feeds = {ph_pure: test_pure, ph_noise: test_noise, ph_crpt: test_crpt}
    test_loss, tb_test_loss = sess.run([loss,_test_loss], feed_dict=test_feeds)

    writer.add_summary(tb_train_loss, epoch)
    writer.add_summary(tb_test_loss, epoch)
    print("Epoch : %03d/%03d  Train_loss : %.7f  Test_loss : %.7f" 
          % (epoch+1, training_epochs, train_loss, test_loss))   
        
    # Display
    if (epoch+1) % display_step == 0:
        train_gt_pure = train_img[train_disp_idx]    # pure image
        train_gt_noise = noise_batch(n_plot)    # random noise
        train_gt_crpt = occl(train_gt_pure,train_gt_noise)   # corrupted image
        train_gt_feeds = {ph_pure: train_gt_pure, ph_noise: train_gt_noise, ph_crpt: train_gt_crpt}
        
        test_gt_pure = test_img[test_disp_idx]    # pure image
        test_gt_noise = noise_batch(n_plot)    # random noise
        test_gt_crpt = occl(test_gt_pure,test_gt_noise)   # corrupted image
        test_gt_feeds = {ph_pure: test_gt_pure, ph_noise: test_gt_noise, ph_crpt: test_gt_crpt}
        
        # generated images
        train_gen_pure, train_gen_noise, train_gen_crpt = sess.run([core_gen, shell2_gen, full_gen], \
                                        feed_dict=train_gt_feeds)  # 784-d vector
        test_gen_pure, test_gen_noise, test_gen_crpt = sess.run([core_gen, shell2_gen, full_gen], \
                                        feed_dict=test_gt_feeds)  # 784-d vector
        
        # plotting results from training data
        fig, axes = plt.subplots(nrows=4, ncols=n_plot, figsize=(10,2*n_plot))   # displaying 4*n_plot images
        plt.setp(axes, xticks=np.arange(0,27,7), yticks=np.arange(0,27,7)) 
        for j in range(n_plot):
            train_disp_gt_crpt = np.reshape(train_gt_crpt[j], [28,28])    # 28x28
            axes[0, j].imshow(train_disp_gt_crpt, cmap='gray')   
            axes[0, j].set(ylabel='gt_crpt')
            axes[0, j].label_outer()
            
            train_disp_gen_pure = np.reshape(train_gen_pure[j], [28,28])    # 28x28
            axes[1, j].imshow(train_disp_gen_pure, cmap='gray')   
            axes[1, j].set(ylabel='gen_pure')
            axes[1, j].label_outer()
            
            train_disp_gen_noise = np.reshape(train_gen_noise[j], [28,28])    # 28x28
            axes[2, j].imshow(train_disp_gen_noise, cmap='gray')   
            axes[2, j].set(ylabel='gen_noise')
            axes[2, j].label_outer()
            
            train_disp_gen_crpt = np.reshape(train_gen_crpt[j], [28,28])    # 28x28
            axes[3, j].imshow(train_disp_gen_crpt, cmap='gray')   
            axes[3, j].set(ylabel='gen_crpt')
            axes[3, j].label_outer()
            
#             train_disp_gt_pure = np.reshape(train_gt_pure[j], [28,28])
#             axes[0, j].imshow(train_disp_gt_pure, cmap='gray')
#             axes[0, j].set(ylabel='gt_pure')
#             axes[0, j].label_outer()
            
#             train_disp_gt_noise = np.reshape(train_gt_noise[j], [28,28])    # 28x28
#             axes[2, j].imshow(train_disp_gt_noise, cmap='gray')   
#             axes[2, j].set(ylabel='gt_noise')
#             axes[2, j].label_outer()
                    
        plt.savefig(outputdir+'/train/epoch %03d' %(epoch+1))    
        plt.close(fig)
        
        # plotting results from testing data
        fig, axes = plt.subplots(nrows=4, ncols=n_plot, figsize=(10,2*n_plot))   # displaying 4*n_plot images
        plt.setp(axes, xticks=np.arange(0,27,7), yticks=np.arange(0,27,7)) 
        for k in range(n_plot):
            test_disp_gt_crpt = np.reshape(test_gt_crpt[k], [28,28])    # 28x28
            axes[0, k].imshow(test_disp_gt_crpt, cmap='gray')   
            axes[0, k].set(ylabel='gt_crpt')
            axes[0, k].label_outer()
            
            test_disp_gen_pure = np.reshape(test_gen_pure[k], [28,28])    # 28x28
            axes[1, k].imshow(test_disp_gen_pure, cmap='gray')   
            axes[1, k].set(ylabel='gen_pure')
            axes[1, k].label_outer()           
            
            test_disp_gen_noise = np.reshape(test_gen_noise[k], [28,28])    # 28x28
            axes[2, k].imshow(test_disp_gen_noise, cmap='gray')   
            axes[2, k].set(ylabel='gen_noise')
            axes[2, k].label_outer()
            
            test_disp_gen_crpt = np.reshape(test_gen_crpt[k], [28,28])    # 28x28
            axes[3, k].imshow(test_disp_gen_crpt, cmap='gray')   
            axes[3, k].set(ylabel='gen_crpt')
            axes[3, k].label_outer()
            
#             test_disp_gt_pure = np.reshape(test_gt_pure[k], [28,28])
#             axes[0, k].imshow(test_disp_gt_pure, cmap='gray')
#             axes[0, k].set(ylabel='gt_pure')
#             axes[0, k].label_outer()
            
#             test_disp_gt_noise = np.reshape(test_gt_noise[k], [28,28])    # 28x28
#             axes[2, k].imshow(test_disp_gt_noise, cmap='gray')   
#             axes[2, k].set(ylabel='gt_noise')
#             axes[2, k].label_outer()
                    
        plt.savefig(outputdir+'/test/epoch %03d' %(epoch+1))    
        plt.close(fig)
        
        # Save
        if (epoch+1) % FLAGS.save_step ==0:
            savename = savedir+"/net-"+str(epoch+1)+".ckpt"
            saver.save(sess, savename)
            print("[%s] SAVED" % (savename))

print("Optimization Finished")

Epoch : 001/500  Train_loss : 5454.3647461  Test_loss : 5012.3193359
Epoch : 002/500  Train_loss : 4255.8417969  Test_loss : 3928.7670898
Epoch : 003/500  Train_loss : 3372.7214355  Test_loss : 3534.4455566
Epoch : 004/500  Train_loss : 3281.3063965  Test_loss : 3327.9497070
Epoch : 005/500  Train_loss : 2920.5751953  Test_loss : 2762.7153320
[./nets/net-5.ckpt] SAVED
Epoch : 006/500  Train_loss : 2775.1901855  Test_loss : 2521.0156250
Epoch : 007/500  Train_loss : 2335.6823730  Test_loss : 2475.6821289
Epoch : 008/500  Train_loss : 2889.8679199  Test_loss : 2621.0732422
Epoch : 009/500  Train_loss : 2148.3845215  Test_loss : 2323.3823242
Epoch : 010/500  Train_loss : 2199.6513672  Test_loss : 2189.1513672
[./nets/net-10.ckpt] SAVED
Epoch : 011/500  Train_loss : 2280.5869141  Test_loss : 2218.4072266
Epoch : 012/500  Train_loss : 2221.2485352  Test_loss : 2119.8720703
Epoch : 013/500  Train_loss : 1957.3471680  Test_loss : 2038.6566162
Epoch : 014/500  Train_loss : 1915.5449219  Test_l

Epoch : 112/500  Train_loss : 1077.6881104  Test_loss : 1032.1082764
Epoch : 113/500  Train_loss : 1102.9626465  Test_loss : 1272.9294434
Epoch : 114/500  Train_loss : 904.7679443  Test_loss : 936.6815796
Epoch : 115/500  Train_loss : 1098.3111572  Test_loss : 1179.1013184
[./nets/net-115.ckpt] SAVED
Epoch : 116/500  Train_loss : 1182.3854980  Test_loss : 971.2183228
Epoch : 117/500  Train_loss : 1158.1480713  Test_loss : 1091.7724609
Epoch : 118/500  Train_loss : 1034.0587158  Test_loss : 941.5605469
Epoch : 119/500  Train_loss : 1029.7268066  Test_loss : 1029.6093750
Epoch : 120/500  Train_loss : 971.2425537  Test_loss : 1046.6250000
[./nets/net-120.ckpt] SAVED
Epoch : 121/500  Train_loss : 1156.6398926  Test_loss : 1107.2523193
Epoch : 122/500  Train_loss : 1066.5659180  Test_loss : 1189.3062744
Epoch : 123/500  Train_loss : 1046.5021973  Test_loss : 1066.5864258
Epoch : 124/500  Train_loss : 1150.6079102  Test_loss : 1110.7536621
Epoch : 125/500  Train_loss : 1015.1950684  Test_los

Epoch : 224/500  Train_loss : 879.1410522  Test_loss : 1033.1687012
Epoch : 225/500  Train_loss : 885.6442871  Test_loss : 865.0101318
[./nets/net-225.ckpt] SAVED
Epoch : 226/500  Train_loss : 897.6739502  Test_loss : 1002.0284424
Epoch : 227/500  Train_loss : 779.9050293  Test_loss : 772.3883667
Epoch : 228/500  Train_loss : 824.7344360  Test_loss : 897.4357910
Epoch : 229/500  Train_loss : 927.2681274  Test_loss : 893.1401367
Epoch : 230/500  Train_loss : 926.9558105  Test_loss : 839.1156616
[./nets/net-230.ckpt] SAVED
Epoch : 231/500  Train_loss : 863.4737549  Test_loss : 952.4473877
Epoch : 232/500  Train_loss : 844.1868896  Test_loss : 1030.5745850
Epoch : 233/500  Train_loss : 882.4113770  Test_loss : 906.6337891
Epoch : 234/500  Train_loss : 756.8988037  Test_loss : 973.7802734
Epoch : 235/500  Train_loss : 911.7322998  Test_loss : 868.0529175
[./nets/net-235.ckpt] SAVED
Epoch : 236/500  Train_loss : 795.0942383  Test_loss : 902.7725830
Epoch : 237/500  Train_loss : 870.0540161 

Epoch : 337/500  Train_loss : 812.5161743  Test_loss : 892.0006104
Epoch : 338/500  Train_loss : 809.5455322  Test_loss : 741.8823242
Epoch : 339/500  Train_loss : 811.6099243  Test_loss : 875.9949951
Epoch : 340/500  Train_loss : 763.8227539  Test_loss : 778.9764404
[./nets/net-340.ckpt] SAVED
Epoch : 341/500  Train_loss : 776.5987549  Test_loss : 895.3458862
Epoch : 342/500  Train_loss : 710.0176392  Test_loss : 781.1658936
Epoch : 343/500  Train_loss : 785.3560791  Test_loss : 777.9759521
Epoch : 344/500  Train_loss : 602.7263794  Test_loss : 834.8350830
Epoch : 345/500  Train_loss : 691.8328857  Test_loss : 958.4066162
[./nets/net-345.ckpt] SAVED
Epoch : 346/500  Train_loss : 811.8183594  Test_loss : 886.8152466
Epoch : 347/500  Train_loss : 878.2510376  Test_loss : 791.9460449
Epoch : 348/500  Train_loss : 724.7451782  Test_loss : 717.8255615
Epoch : 349/500  Train_loss : 751.8057861  Test_loss : 817.1591797
Epoch : 350/500  Train_loss : 844.8973999  Test_loss : 876.9810181
[./net

[./nets/net-450.ckpt] SAVED
Epoch : 451/500  Train_loss : 715.9310913  Test_loss : 829.0169678
Epoch : 452/500  Train_loss : 775.9942627  Test_loss : 807.6539917
Epoch : 453/500  Train_loss : 637.9610596  Test_loss : 622.7006836
Epoch : 454/500  Train_loss : 784.5960693  Test_loss : 706.9374390
Epoch : 455/500  Train_loss : 719.5009766  Test_loss : 655.6868286
[./nets/net-455.ckpt] SAVED
Epoch : 456/500  Train_loss : 684.6130981  Test_loss : 757.6189575
Epoch : 457/500  Train_loss : 681.7459106  Test_loss : 653.5174561
Epoch : 458/500  Train_loss : 663.4273682  Test_loss : 668.8614502
Epoch : 459/500  Train_loss : 579.7672119  Test_loss : 818.6263428
Epoch : 460/500  Train_loss : 629.8052979  Test_loss : 723.0996704
[./nets/net-460.ckpt] SAVED
Epoch : 461/500  Train_loss : 602.3751831  Test_loss : 650.1722412
Epoch : 462/500  Train_loss : 641.3475952  Test_loss : 774.3695068
Epoch : 463/500  Train_loss : 905.0561523  Test_loss : 700.2164307
Epoch : 464/500  Train_loss : 833.9270020  Te